In [ ]:
%tensorflow_version 1.x
from keras.models import Sequential
from keras.layers import Dense
from __future__ import print_function
import tensorflow as tf

from keras.layers import Activation
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects
from tensorflow.python.keras.backend import get_session

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
import keras
import tensorflow
batch = 32

TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
@tf.function
def custom_activation(x):
   global batch
   return tf.where(tf.less_equal(x, 0.5),tf.ones((32,7684)),tf.zeros((32,7684)))

get_custom_objects().update({'custom_activation': Activation(custom_activation)})

In [ ]:
def define_discriminator(n_inputs=7684):
  model = Sequential()
  model.add(Dense(7684,activation="relu",input_dim=7684))
  model.add(Dense(200,activation="sigmoid"))
  model.add(Dense(200,activation="relu"))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(
  optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy'],
  )
  model.load_weights("/content/drive/MyDrive/dataset master/model1.h5")
	#model =  tensorflow.keras.models.load_model("/content/drive/MyDrive/dataset master/model.h5")
  return model

In [ ]:
def define_generator(latent_dim, n_outputs=7684):
  model = Sequential()
  model.add(Dense(7684, activation="relu",input_dim=7684))
  model.add(Dense(3000,activation="softmax"))
  model.add(Dense(200,activation="tanh"))
  model.add(Dense(7684, activation='sigmoid'))
  model.compile(
  optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy'],
  )
  #model.add(Activation(custom_activation, name='SpecialActivation'))
  return model

In [ ]:
def define_gan(generator, discriminator):
	# make weights in the discriminator not trainable
	discriminator.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(generator)
	# add the discriminator
	model.add(discriminator)
	# compile model
	model.compile(loss='binary_crossentropy', optimizer='adam')
	return model

In [ ]:
def generate_real_samples(n):
  dataset = pd.read_csv("/content/drive/MyDrive/dataset master/benign.csv",skiprows=0,nrows=n).fillna(0)
  X = dataset.drop("40",axis=1)
  X = X.to_numpy()
	# generate class labels
  y = zeros((n, 1))
  return X, y

In [ ]:
def generate_real_samples_for_evaluation(n):
  dataset = pd.read_csv("/content/drive/MyDrive/dataset master/benign.csv",skiprows=5344,nrows=n).fillna(0)
  X = dataset.drop("10",axis=1)
  X = X.to_numpy()
	# generate class labels
  y = zeros((n, 1))
  return X, y

In [ ]:
def generate_latent_points(latent_dim, n):
  dataset = pd.read_csv("/content/drive/MyDrive/dataset master/malware.csv",skiprows=0,nrows=n).fillna(0)
  X = dataset.drop("100",axis=1)
  X = X.to_numpy()
  return X

In [ ]:
def generate_latent_points_for_evaluation(latent_dim, n):
  dataset = pd.read_csv("/content/drive/MyDrive/dataset master/malware.csv",skiprows=5344,nrows=n).fillna(0)
  X = dataset.drop("100",axis=1)
  X = X.to_numpy()
  return X

In [ ]:
def generate_fake_samples(generator, latent_dim, n):
  global batch
  # generate points in latent space
  x_input = generate_latent_points(latent_dim, n)
  # predict outputs
  X = generator.predict(x_input)
  for i in range(len(X)):
    for j in range(7684):
      if X[i][j] < 0.5 and x_input[i][j] == 1:
          X[i][j] = 1
      elif X[i][j] < 0.5 and x_input[i][j] == 0:
          X[i][j] = 0
      elif X[i][j] > 0.5:
          X[i][j] = 1
  # create class labels
  y = ones((len(X), 1))
  batch = len(X)
  return X, y

In [ ]:
def generate_fake_samples_for_evaluation(generator, latent_dim, n = 1344):
  global batch
  # generate points in latent space
  x_input = generate_latent_points_for_evaluation(latent_dim, n)
  # predict outputs
  X = generator.predict(x_input)
  for i in range(len(X)):
    for j in range(7684):
      if X[i][j] < 0.5 and x_input[i][j] == 1:
          X[i][j] = 1
      elif X[i][j] < 0.5 and x_input[i][j] == 0:
          X[i][j] = 0
      elif X[i][j] > 0.5:
          X[i][j] = 1

  # create class labels
  y = ones((len(X), 1))
  batch = len(X)
  return X, y

In [ ]:
def summarize_performance(epoch, generator, discriminator, latent_dim, n=1344):
	# prepare real samples
  x_real, y_real = generate_real_samples_for_evaluation(n)
	# evaluate discriminator on real examples
  _, acc_real = discriminator.evaluate(x_real, y_real, verbose=0)
	# prepare fake examples
  x_fake, y_fake = generate_fake_samples_for_evaluation(generator, latent_dim, n)
	# evaluate discriminator on fake examples
  _, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
  # summarize discriminator performance
  generator.save("model.h5")
  print("epoch : "+str(epoch)+"acc_real : "+str(acc_real)+"acc_fake : "+str(acc_fake))

In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, latent_dim, n_epochs=10000, n_batch=2048, n_eval=2000):
	# determine half the size of one batch, for updating the discriminator
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# prepare real samples
		x_real, y_real = generate_real_samples(half_batch)
		# prepare fake examples
		x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)

    # update discriminator
		d_model.train_on_batch(x_real, y_real)
		d_model.train_on_batch(x_fake, y_fake)

    # prepare points in latent space as input for the generator
		x_gan = generate_latent_points(latent_dim, n_batch)
		# create inverted labels for the fake samples
		y_gan = zeros((n_batch, 1))
		# update the generator via the discriminator's error
		gan_model.train_on_batch(x_gan, y_gan)
		# evaluate the model every n_eval epochs
		if (i+1) % n_eval == 0:
			summarize_performance(i, g_model, d_model, latent_dim)

In [ ]:
# size of the latent space
latent_dim = 7684
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# train model
train(generator, discriminator, gan_model, latent_dim,n_epochs=10,n_eval=1,n_batch=10688)

In [ ]:
  ## TEST GENERATOR ##
  Gmodel = Sequential()
  Gmodel.add(Dense(7684, activation="relu",input_dim=7684))
  Gmodel.add(Dense(3000,activation="softmax"))
  Gmodel.add(Dense(200,activation="tanh"))
  Gmodel.add(Dense(7684, activation='sigmoid'))
  Gmodel.compile(
  optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy'],
  )
  Gmodel.load_weights("model.h5")

In [ ]:
model = Sequential()
model.add(Dense(7684,activation="relu",input_dim=7684))
model.add(Dense(200,activation="sigmoid"))
model.add(Dense(200,activation="relu"))
model.add(Dense(1,activation='sigmoid'))
model.compile(
optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy'],
)
model.load_weights("/content/drive/MyDrive/dataset master/model1.h5")

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/dataset master/malware.csv",skiprows=5344).fillna(0)
X = dataset.drop("100",axis=1)
X = X.to_numpy()

In [ ]:
prediction = model.predict(X)
y_pred = np.argmax(prediction, axis=1)
print(y_pred)

In [ ]:
generated = Gmodel.predict(X)
for i in range(len(X)):
    for j in range(7684):
      if generated[i][j] < 0.5 and X[i][j] == 1:
          generated[i][j] = 1
      elif generated[i][j] < 0.5 and X[i][j] == 0:
          generated[i][j] = 0
      elif generated[i][j] > 0.5:
          generated[i][j] = 1

Generated_predection = model.predict(generated)
y_predG = np.argmax(Generated_predection, axis=1)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
print(accuracy_score(y_pred, y_predG))
print(confusion_matrix(y_pred, y_predG))
print(classification_report(y_pred, y_predG))

In [ ]:
print(accuracy_score(y_pred, np.ones((len(y_pred),1))))
print(confusion_matrix(y_pred,  np.ones((len(y_pred),1))))
print(classification_report(y_pred,  np.ones((len(y_pred),1))))

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/dataset master/malware.csv").fillna(0)


In [ ]:
y = dataset["100"]
X = dataset.drop("100", axis=1)

In [ ]:
train_images, test_images, train_labels, test_labels = train_test_split(X, y, test_size=0.0001)
train_images = train_images.to_numpy()
test_images = test_images.to_numpy()
batch = len(test_images)

In [ ]:

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)
print(model.summary())

In [ ]:
zero = 0
un = 0
prediction = model.predict(test_images)[1]
for i in range(7684):
  if prediction[i] == test_images[1][i]:
     zero = zero + 1
  else:
    un = un + 1
print(zero)
print(un)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(7684, activation = 'sigmoid',input_dim=7684))
model.add(MyCustomLayer(32, input_shape = (7684,)))

model.summary()
print(model.predict(X)[0])